In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import sys
sys.path.append("/Users/alessiorussointroito/Documents/GitHub/Structural-Perturbation-Method")

#from SPM_fast import SPM
from BiSPM import BiSPM

import similaripy as sim

In [26]:
import math

In [23]:
np.random.seed(42)

In [2]:
df = pd.read_csv(
    "/Users/alessiorussointroito/Downloads/Telegram Desktop/recommender-system-2020-challenge-polimi/data_train.csv")

le = LabelEncoder()
df['new_col'] = le.fit_transform(df.col)

row_size = len(df.row.unique())
col_size = len(le.classes_)

In [8]:
df = df[["row", "new_col", "data"]].rename(columns={"row":"user_id", "new_col":"item_id"})

In [9]:
df

,user_id,item_id,data
0,0,9654,1.0
1,0,18640,1.0
2,1,2567,1.0
3,1,7181,1.0
4,1,16334,1.0
...,...,...,...
113263,7945,2385,1.0
113264,7945,11793,1.0
113265,7945,20490,1.0
113266,7946,8329,1.0


# Cross validation

In [10]:
group = df.groupby("user_id")['item_id'].apply(lambda x: list(x)).reset_index()

In [16]:
n_users = group.user_id.shape[0]
users_per_fold = int(n_users / 5)

In [27]:
hold_perc = []
holdout_perc = 0.2
for l in group.item_id:
    hold_perc.append(math.ceil(len(l)*holdout_perc))
group['holdout'] = hold_perc

In [28]:
res = []
for l, h in zip(group.item_id, group.holdout):
    res.append(np.random.choice(l, h))
group['target'] = res

In [30]:
group

,user_id,item_id,holdout,target
0,0,"[9654, 18640]",1,[9654]
1,1,"[2567, 7181, 16334, 16964, 17359, 19300]",2,"[16964, 17359]"
2,2,"[18516, 20297]",1,[18516]
3,3,"[17942, 22073]",1,[17942]
4,4,"[462, 6641, 9775, 13123, 18194, 18999, 19012]",2,"[18194, 18194]"
...,...,...,...,...
7942,7942,"[7505, 19278, 20756, 21344, 23382]",1,[20756]
7943,7943,"[7554, 7559, 11255]",1,[11255]
7944,7944,"[176, 252, 258, 372, 402, 413, 996, 1189, 1462...",32,"[11990, 4843, 258, 12486, 8404, 11478, 19010, ..."
7945,7945,"[791, 2385, 11793, 20490]",1,[20490]


In [36]:
fold1, fold2, fold3, fold4, fold5 = group.iloc[:users_per_fold, :], \
                                    group.iloc[users_per_fold:(users_per_fold*2), :],\
                                    group.iloc[(users_per_fold*2):(users_per_fold*3), :],\
                                    group.iloc[(users_per_fold*3):(users_per_fold*4), :],\
                                    group.iloc[(users_per_fold*4):, :]

In [34]:
fold1

,user_id,item_id,holdout,target
0,0,"[9654, 18640]",1,[9654]
1,1,"[2567, 7181, 16334, 16964, 17359, 19300]",2,"[16964, 17359]"
2,2,"[18516, 20297]",1,[18516]
3,3,"[17942, 22073]",1,[17942]
4,4,"[462, 6641, 9775, 13123, 18194, 18999, 19012]",2,"[18194, 18194]"
...,...,...,...,...
1584,1584,"[6342, 10315, 15725, 15759, 17839, 18100]",2,"[6342, 15759]"
1585,1585,"[171, 568, 658, 1150, 2831, 3085, 3349, 3981, ...",7,"[3981, 5447, 22843, 9291, 11093, 20125, 3349]"
1586,1586,"[15483, 19023]",1,[19023]
1587,1587,"[339, 13900, 15312, 24644]",1,[339]


In [42]:
count = 0
for i in [e for x in fold1.target for e in x]:
    count += 1
count

4860

In [55]:
def get_csr_matrix(df, row_col, item_col):
    row_idx = []
    item_idx = []

    for r, l in zip(df[row_col], df[item_col]):
        for e in l:
            row_idx.append(r)
            item_idx.append(e)
            
    print(len(row_idx))
    print(len(item_idx))
    print(len(np.ones(len(row_idx))))
            
    assert(len(row_idx) == len(item_idx)), "Error, different lens"
    matrix = sps.csr_matrix((np.ones(len(row_idx)), (row_idx, item_idx)), shape=(row_size, col_size))
    return matrix
    
def get_train_test_urm(folds, test):
    #pd.concat(folds)
    
    test_urm = get_csr_matrix(test, "user_id", "target")
    return test_urm

In [56]:
get_train_test_urm([fold2, fold3, fold4, fold5], fold1).nnz

4860
4860
4860


4501

TypeError: 'numpy.ndarray' object is not callable